In [ ]:
import numpy as np
from scipy.integrate import quad

class Heston:
    def __init__(self, S0, v0, r, k, theta, eta, rho):
        self.S0 = S0
        self.v0 = v0
        self.r = r
        self.k = k
        self.theta = theta
        self.eta = eta
        self.rho = rho
        self.gamma = eta**2/2.0
    def P(self, j, x, tau):
        return 0.5+1.0/np.pi*(quad(self.integrand, 0.0, np.inf, args = (j, x, self.v0, tau)))[0]
    def integrand(self, u, j, x, v, tau):
        return np.real(np.exp(self.C(j, u, tau)*self.theta+self.D(j, u, tau)*v+1j*u*x)/(u*1j))
    def C(self, j, u, tau):
        g = self.rminus(j, u)/self.rplus(j, u)
        return self.k*(self.rminus(j,u)*tau-2.0/self.eta**2*np.log((1.0-g*np.exp(-self.d(j,u)*tau))/(1.0-g)))
    def d(self, j, u):
        return np.sqrt(self.beta(j,u)**2-4*self.alpha(j,u)*self.gamma)
    def rminus(self, j, u):
        return (self.beta(j, u)-self.d(j,u))/(2*self.gamma)
    def rplus(self, j, u):
        return (self.beta(j, u) + self.d(j, u))/(2*self.gamma)
    def beta(self, j, u):
        return self.k - self.rho*self.eta*j - self.rho*self.eta*u*1j
    def alpha(self, j, u):
        return -u**2/2-u*1j/2+j*u*1j
    def D(self, j, u, tau):
        g = self.rminus(j, u)/self.rplus(j, u)
        return self.rminus(j,u)*(1.0-np.exp(-self.d(j,u)*tau))/(1.0-g*np.exp(-self.d(j,u)*tau))
    def callprice(self, strike, tau):
        B = np.exp(-self.r*tau)
        F = self.S0/B
        x = np.log(F/strike)
        return B*(F*self.P(1,x,tau)-strike*self.P(0,x,tau))
#     def putprice(self, strike, tau):
#         B = np.exp(-self.r*tau)
#         F = self.S0/B
#         x = np.log(F/strike)
#         return B*(strike*self.P(0,-x,tau)-F*self.P(1,-x,tau))
    def putprice_test(self, strike, tau):
        B = np.exp(-self.r*tau)
        F = self.S0/B
        x = np.log(F/strike)
        return self.callprice(strike, tau) - self.S0 + B*strike #put price defined by put-call parity

#defining function HestonOption that calls the above class as req'd

def HestonOption(S0, v0, strike, eta, r, tau, callput, k, theta, rho):
    hestonmodel = Heston(S0, v0, r, k, theta, eta, rho)
    if callput == 1:
        return hestonmodel.callprice(strike, tau)
    elif callput == -1:
        return hestonmodel.putprice_test(strike, tau)
    else:
        raise ValueError("callput must be either 1 (call) or -1 (put)")
        
stock = float(input('Enter the underlying stock price: '))
strike = float(input('Enter the strike price: '))
v0 = float(input('Enter the initial volatility: '))
interest = float(input('Enter continuously compounded interest rate: '))
maturity = float(input('Enter the time to maturity: '))
callput = int(input('Enter 1 for call or -1 for put option: '))
eta = float(input('Enter the volatility of volatility: '))
k = float(input('Enter the speed of mean reversion: '))
theta = float(input('Enter the long-run mean volatility: '))
rho = float(input('Enter the instantaneous correlation: '))
result = HestonOption(stock,v0,strike,eta,interest,maturity,callput,k,theta,rho)
print('The option price is '+str(result))